In [1]:
from astropy.io import ascii
from astropy.table import vstack,Table,Column


In [2]:
galaxies = Table.read('/home/mel/Documents/GZ_HUBBLE/gzh_red_disks/new_ferengi/data/ferengi_candidates_stage_4_936.fits')

In [3]:
len(galaxies)

936

In [8]:
#if there's more than one source detected in sextractor: select source with largest ISOAREA_IMAGE
def trim_table(sextable):
    trimmed_sextable=Table()
    for name in sextable.colnames:
        c0 = Column([x[name] for x in sextable], name=name) 
        trimmed_sextable.add_columns([c0])
    trimmed_sextable.sort('ISOAREA_IMAGE')
    
    while len(trimmed_sextable)>1:
        trimmed_sextable.remove_rows(0)
    
    return trimmed_sextable

#create table of SExtractor measurements
def make_table(galaxies,band):
    simzs=[3,4,5,6,7,8,9,10]

    zero_sources_list = {} #keep track of files with 0 sources detected in SExtractor
    for z in simzs:
        zero_sources_list[z/10.]=[]
    
    for i,gal in enumerate(galaxies):
        objid = gal['dr12objid']
        for simz in simzs:
            fname = 'SEx_output_{}/{}_imout_{}_simz_{}_evo_1.cat'.format(band,objid,band,simz)
            sex_table = Table.read(fname,format='ascii')
        
            if len(sex_table)>1: #multiple sources detected, pick out source with largest area
                sex_table = trim_table(sex_table)
            
            if len(sex_table)==1: #we're good, at least 1 and only 1 source 
                #add some meta to sex tables
                c1 = Column([objid],name='dr12objid')
                c2 = Column([gal['redshift']],name='redshift')
                c3 = Column([simz/10.],name='simz')
                sex_table.add_columns([c1,c2,c3],[0,0,0])
            
            if len(sex_table)==0:
                zero_sources_list[simz/10.].append(objid)#keep track of files where no sources are detected :/
            
            if i==0 and simz==3: #first table! save it for stacking
                stacked_table = sex_table
            else: #stack table onto current sex table
                stacked_table = vstack([stacked_table,sex_table])
            
        if i%50==0:
            print '{} galaxies done'.format(i)
            
    stacked_table.remove_column('NUMBER') #useless column
        
    #add _band to columns for clarity!
    for name in stacked_table.colnames:
        if name != 'dr12objid' and name!= 'redshift' and name!= 'simz':
            stacked_table.rename_column(name, name+'_'+band)


            
    return stacked_table, zero_sources_list

## Create I,V band measurements table

In [9]:
table_I, zero_sources_list_I = make_table(galaxies,'I')

0 galaxies done
50 galaxies done
100 galaxies done
150 galaxies done
200 galaxies done
250 galaxies done
300 galaxies done
350 galaxies done
400 galaxies done
450 galaxies done
500 galaxies done
550 galaxies done
600 galaxies done
650 galaxies done
700 galaxies done
750 galaxies done
800 galaxies done
850 galaxies done
900 galaxies done


In [37]:
table_V, zero_sources_list_V = make_table(galaxies,'V')

0 galaxies done
50 galaxies done
100 galaxies done
150 galaxies done
200 galaxies done
250 galaxies done
300 galaxies done
350 galaxies done
400 galaxies done
450 galaxies done
500 galaxies done
550 galaxies done
600 galaxies done
650 galaxies done
700 galaxies done
750 galaxies done
800 galaxies done
850 galaxies done
900 galaxies done


In [12]:
simzs=[3,4,5,6,7,8,9,10]

for z in simzs:
    print '{} missing at redshfit {}'.format(len(zero_sources_list_I[z/10.]),z/10.)

2 missing at redshfit 0.3
11 missing at redshfit 0.4
36 missing at redshfit 0.5
63 missing at redshfit 0.6
92 missing at redshfit 0.7
128 missing at redshfit 0.8
173 missing at redshfit 0.9
208 missing at redshfit 1.0


In [38]:
for z in simzs:
    print '{} missing at redshfit {}'.format(len(zero_sources_list_V[z/10.]),z/10.)

4 missing at redshfit 0.3
20 missing at redshfit 0.4
46 missing at redshfit 0.5
84 missing at redshfit 0.6
140 missing at redshfit 0.7
185 missing at redshfit 0.8
229 missing at redshfit 0.9
249 missing at redshfit 1.0


In [42]:
all_missing_I = zero_sources_list_I[.3]+ zero_sources_list_I[.4]+ zero_sources_list_I[.5]+zero_sources_list_I[.6]+ zero_sources_list_I[.7]+ zero_sources_list_I[.8]+ zero_sources_list_I[.9]+zero_sources_list_I[1.0]
all_unique_missing_I = list(set(all_missing_I))
all_missing_V = zero_sources_list_V[.3]+ zero_sources_list_V[.4]+ zero_sources_list_V[.5]+zero_sources_list_V[.6]+ zero_sources_list_V[.7]+ zero_sources_list_V[.8]+ zero_sources_list_V[.9]+zero_sources_list_V[1.0]
all_unique_missing_V = list(set(all_missing_V))

In [45]:
table_I.write('ferengi2_I.fits')
table_V.write('ferengi2_V.fits')

In [20]:
# get pictures of galaxies that got ferengied into oblivion
import matplotlib.gridspec as gridspec
import Image
import ImageOps
import matplotlib.pyplot as plt
import urllib, cStringIO

plt.rcParams['text.usetex']=True


322

In [21]:
#get galaxy filename
def image_from_file(objid,z):
    f = '../Ferengi2Sample/Images/{:d}_simz_{:d}_evo_1.jpg'.format(objid,z)
    img = Image.open(f)

    return img
#function for getting galaxy images
def image_from_url(gal):
    
    pixelsize=0.396
    height=500
    width=500

    scale=2.5*gal['petror90_r']/pixelsize/width
    # form wurl to query SDSS web sevice to generate cutout image
    cutOutImageUrl="http://skyservice.pha.jhu.edu/DR12/ImgCutout/getjpeg.aspx?ra=" + str(gal['RA'])
    cutOutImageUrl += "&dec=" + str(gal['DEC']) + "&scale=""" + str(scale) + "&width=" + str(width)
    cutOutImageUrl += "&height=" + str(height)
    
    file = cStringIO.StringIO(urllib.urlopen(cutOutImageUrl).read())
    img = Image.open(file)    
    return img

In [35]:
zs = [3,4,5,6,7,8,9,10]
f = plt.figure(figsize=(25,160))
gs = gridspec.GridSpec(50,9)
gs.update(wspace=0)

for j in range(0,50):
    objid = all_unique_missing[j]
    gal = galaxies[galaxies['dr12objid']==objid][0]
    #gal = galaxies[j]
    objid = gal['dr12objid']
    #first cell: original image
    ax = plt.subplot(gs[j,0])
    plt.imshow(image_from_url(gal))

    plt.tick_params(labelleft='off',labelbottom='off')
    plt.title('z={:.3} (sdss gri)'.format(gal['redshift']),fontsize=15)
    plt.ylabel('{:d}'.format(gal['dr12objid']),fontsize=15)

    #next: ferengified images at z=0.3, 0.6, 1.0
    for i,z in enumerate(zs):
        ax = plt.subplot(gs[j,i+1])
        plt.imshow(image_from_file(gal['dr12objid'],z))
        
        #check if detected or not
        if objid in zero_sources_list_I[z/10.]:
            ax.text(.1,.08,r'X',fontsize=20,color='w',transform=ax.transAxes)

        plt.tick_params(labelleft='off',labelbottom='off')
        plt.title('z={}'.format(z/10.),fontsize=20)
        
plt.savefig('50_undetected_galaxies.pdf')